In [1]:
import pandas as pd
import numpy as np
import os
import json
from PIL import Image
from io import StringIO
os.chdir('/home/danish/VR_PROJECT')

In [ ]:
import os
import google.generativeai as genai

# Set the GOOGLE_API_KEY in the environment first

# Then configure the genai module with the API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Now you can create the GenerativeModel instance
model = genai.GenerativeModel('gemini-2.0-flash-001')

/home/danish/VR_PROJECT/.vr_project/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
listing_file = "dataset/abo-listings/listings/metadata/listings_8.json"
image_metadata = pd.read_csv("dataset/abo-images-small/images/metadata/images.csv")
image_dataset_path = "dataset/abo-images-small/images/small/"

In [4]:
#create output csv file
output_file = f"dataset/VQA-dataset/{listing_file.split('/')[-1].split('.')[0]}_VQA.csv"

In [5]:
def get_listing_lines(listing_file):
    """
    Read the listing file and return a list of lines.
    """
    with open(listing_file, 'r') as file:
        lines = file.readlines()
    return lines





In [6]:
def preprocess_product_json(product_json):
        """
        Preprocess the product JSON to ensure it is in the correct format.
        """
        list_of_keys_to_remove = ['main_image_id','node','other_image_id','spin_id','3dmodel_id']
        # Convert JSON string to dictionary
        product_dict = json.loads(product_json)
        # Remove unnecessary keys
        for key in list_of_keys_to_remove:
            if key in product_dict:
                del product_dict[key]
        
        # Convert dictionary to JSON string with indentation for better readability
        return json.dumps(product_dict, indent=4)
def prompt_for_product(product_json):
        """
        Generate a prompt for the given product JSON.
        """
        product_json = preprocess_product_json(product_json)
        prompt = f"""
        You are a QA dataset generator that creates short, factual, and human-readable question-answer pairs from Amazon product metadata and image. Each question must target a specific field from the metadata and be answerable with a **single word only**.

        Below is the product metadata in structured format. Generate **5 to 10 diverse QA pairs**, where:
        - Each question is clear and unambiguous.
        - Each answer is strictly a **single word** (no phrases, no multi-word answers).
        - Avoid repeating the same field.
        - Prefer commonly relevant fields like: brand, bullet_points, color, material, product type, model name, style, fabric type, finish type, pattern, item shape, product description and color code.
        - Questions should be such a way that they can be answered by looking at the image.
        - The output should be in CSV format with columns: question, answer.

        If a value is not meaningful or not present, skip that field. Ensure that QA pairs are diverse and aligned with the data provided.

        ---
        {product_json}
        ---
        """
        
        return prompt

In [9]:
def get_images_paths(image_ids):
    """
    Get the paths of images based on their IDs.
    """
    image_paths = []
    for image_id in image_ids:
        image_path = image_metadata[image_metadata['image_id'] == image_id]['path'].values
        if len(image_path) > 0:
            if os.path.exists(f"{image_dataset_path}/{image_path[0]}"):
                image_paths.append(f"{image_path[0]}")
    if len(image_paths) > 2:
        image_paths = image_paths[:2]
    return image_paths

def generate_VQA(prompt, image_path):
    img = Image.open(f"{image_dataset_path}/{image_path}")
    img = img.convert("RGB")
    # Generate the VQA using the model
    response = model.generate_content([prompt, img])
    # Extract the generated text from the response
    generated_text = response.text
    #read csv from the generated text
    csv_data = pd.read_csv(StringIO(generated_text.strip("`").replace("csv\n", "", 1).strip()))
    return csv_data

def get_VQA_for_product(product_json):
    df = pd.DataFrame(columns=["image_path","question", "answer"])
    list_of_image_ids = []
    
    prompt = prompt_for_product(product_json)
    product_dict = json.loads(product_json)
    if "main_image_id" in product_dict.keys():
        list_of_image_ids.append(product_dict['main_image_id'])
    if "other_image_id" in product_dict.keys():
        # Check if the key exists in the dictionary
        if isinstance(product_dict['other_image_id'], list):
            # If it's a list, extend it to the list_of_image_ids
            list_of_image_ids.extend(product_dict['other_image_id'])
        else:
            # If it's not a list, append it directly
            list_of_image_ids.append(product_dict['other_image_id'])
    image_paths = get_images_paths(list_of_image_ids)

    # print(f"Image paths: {image_paths}")
    # Generate the VQA using the model
    if len(image_paths) == 0:
        print("No images found for this product.")
        return df
    for image_path in image_paths:
        # print(f"Generating VQA for image: {image_path}")
        # Generate the VQA using the model
        csv_data = generate_VQA(prompt, image_path)
        # Append the generated data to the dataframe
        csv_data['image_path'] = image_path
        df = pd.concat([df, csv_data], ignore_index=True)
    return df



    
    
    

In [10]:
import time
from tqdm import tqdm
print(output_file)
if not os.path.exists(output_file):
    print(f"Output file {output_file} does not exist. Creating a new one.")
    output_df = pd.DataFrame(columns=["image_path","question", "answer"])
else:
    output_df = pd.read_csv(output_file)


lines = get_listing_lines(listing_file)

# subset_lines = lines[556:]  # to process a subset of lines
start_index = 7051
try:
    for i in tqdm(range(start_index, len(lines)), initial=start_index, total=len(lines)):
        line = lines[i]
        if "\"en_" not in line:
            continue
        try:
            df = get_VQA_for_product(line)
        except Exception as e:
            print(f"Error processing line {i}: {e}")
            if "Resource has been exhausted" in str(e):
                print("Resource has been exhausted. Please try again later.")
                time.sleep(300)
                try:
                    df = get_VQA_for_product(line)
                except Exception as e:
                    time.sleep(900)
                    df = get_VQA_for_product(line)
            else:
                continue
        output_df = pd.concat([output_df, df], ignore_index=True)
        if i % 50 == 0:
            # Save the output DataFrame to a CSV file every 50 iterations
            output_df.to_csv(output_file, index=False)
            print(f"Progress saved at line {i}")
except Exception as e:
    print(f"An error occurred: {e}")
    # Handle the exception as needed
    pass
# Save the output DataFrame to a CSV file
output_df.to_csv(output_file, index=False)
print(f"VQA dataset saved to {output_file}") 
print(f"Progress saved at line {i}")


dataset/VQA-dataset/listings_8_VQA.csv


 77%|███████▋  | 7101/9232 [02:20<1:42:37,  2.89s/it]

Progress saved at line 7100


 78%|███████▊  | 7171/9232 [05:26<1:34:06,  2.74s/it]

No images found for this product.


...

Progress saved at line 9150


100%|█████████▉| 9201/9232 [1:43:13<01:31,  2.94s/it]

Progress saved at line 9200


100%|██████████| 9232/9232 [1:44:39<00:00,  2.88s/it]

VQA dataset saved to dataset/VQA-dataset/listings_8_VQA.csv
Progress saved at line 9231
